<a href="https://colab.research.google.com/github/Alex-Jung-HB/0719_python_YOLO11-vs-YOLO12/blob/main/0719_python_YOLO11_vs_YOLO12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Top and bottom video combiner to compare YOLO11 and YOLO12

In [2]:
# ========================================
# SIMPLE VIDEO COMBINER - TOP/BOTTOM LAYOUT
# Works with both GUI and Command Line!
# ========================================

import os
import sys

# Try to import moviepy for video processing
try:
    from moviepy.editor import VideoFileClip, clips_array
    print("✅ MoviePy loaded successfully!")
except ImportError:
    print("❌ Error: MoviePy not found!")
    print("📦 Please install it using: pip install moviepy")
    exit()

# Try to import tkinter for GUI, but make it optional
try:
    import tkinter as tk
    from tkinter import filedialog, messagebox
    GUI_AVAILABLE = True
    print("✅ GUI mode available")
except ImportError:
    GUI_AVAILABLE = False
    print("⚠️ GUI not available, using command-line mode")

# ========================================
# COMMAND LINE VIDEO COMBINER
# ========================================

def combine_videos_cli(top_video_path, bottom_video_path, output_path):
    """
    Command-line function to combine two videos

    Args:
        top_video_path (str): Path to the top video file
        bottom_video_path (str): Path to the bottom video file
        output_path (str): Path where to save the combined video
    """

    try:
        print("🎬 Starting video combination process...")

        # Check if input files exist
        if not os.path.exists(top_video_path):
            print(f"❌ Error: Top video file not found: {top_video_path}")
            return False

        if not os.path.exists(bottom_video_path):
            print(f"❌ Error: Bottom video file not found: {bottom_video_path}")
            return False

        # Load videos
        print(f"📂 Loading top video: {os.path.basename(top_video_path)}")
        top_clip = VideoFileClip(top_video_path)

        print(f"📂 Loading bottom video: {os.path.basename(bottom_video_path)}")
        bottom_clip = VideoFileClip(bottom_video_path)

        # Get durations
        top_duration = top_clip.duration
        bottom_duration = bottom_clip.duration

        print(f"⏱️ Top video duration: {top_duration:.2f} seconds")
        print(f"⏱️ Bottom video duration: {bottom_duration:.2f} seconds")

        # Use shorter duration
        final_duration = min(top_duration, bottom_duration)
        print(f"🎯 Using duration: {final_duration:.2f} seconds")

        # Trim videos to same duration
        print("✂️ Trimming videos to same duration...")
        top_clip_trimmed = top_clip.subclip(0, final_duration)
        bottom_clip_trimmed = bottom_clip.subclip(0, final_duration)

        # Combine videos vertically
        print("🔄 Combining videos...")
        combined_video = clips_array([
            [top_clip_trimmed],      # Top row
            [bottom_clip_trimmed]    # Bottom row
        ])

        # Save the result
        print(f"💾 Saving combined video to: {output_path}")
        combined_video.write_videofile(
            output_path,
            codec='libx264',
            audio_codec='aac'
        )

        # Cleanup
        print("🧹 Cleaning up...")
        top_clip.close()
        bottom_clip.close()
        top_clip_trimmed.close()
        bottom_clip_trimmed.close()
        combined_video.close()

        print("✅ Video combined successfully!")
        return True

    except Exception as error:
        print(f"❌ Error: {str(error)}")
        return False

def run_command_line_mode():
    """Run the application in command-line mode"""

    print("=" * 60)
    print("🎬 VIDEO COMBINER - COMMAND LINE MODE")
    print("=" * 60)

    # Get input files from user
    while True:
        top_video = input("📹 Enter path to TOP video file: ").strip()
        if os.path.exists(top_video):
            break
        print("❌ File not found! Please try again.")

    while True:
        bottom_video = input("📹 Enter path to BOTTOM video file: ").strip()
        if os.path.exists(bottom_video):
            break
        print("❌ File not found! Please try again.")

    # Get output path
    output_video = input("💾 Enter output path (e.g., combined_video.mp4): ").strip()
    if not output_video:
        output_video = "combined_video.mp4"

    # Add .mp4 extension if not present
    if not output_video.endswith(('.mp4', '.avi', '.mov')):
        output_video += '.mp4'

    print("\n" + "=" * 60)
    print("📋 SUMMARY:")
    print(f"   Top video: {top_video}")
    print(f"   Bottom video: {bottom_video}")
    print(f"   Output: {output_video}")
    print("=" * 60)

    # Confirm before processing
    confirm = input("Proceed? (y/n): ").strip().lower()
    if confirm in ['y', 'yes']:
        success = combine_videos_cli(top_video, bottom_video, output_video)
        if success:
            print(f"\n🎉 Success! Combined video saved as: {output_video}")
        else:
            print("\n❌ Failed to combine videos!")
    else:
        print("❌ Operation cancelled.")

# ========================================
# GUI VIDEO COMBINER (Only if GUI available)
# ========================================

if GUI_AVAILABLE:
    class SimpleCombiner:
        """GUI version of the video combiner"""

        def __init__(self):
            # Check if display is available before creating GUI
            try:
                self.root = tk.Tk()
                self.root.withdraw()  # Hide window temporarily
                self.root.destroy()   # Test if GUI works
            except tk.TclError as e:
                print(f"❌ GUI Error: {e}")
                print("🔄 Falling back to command-line mode...")
                raise

            # If we get here, GUI works - create the real window
            self.root = tk.Tk()
            self.root.title("🎬 Simple Video Combiner")
            self.root.geometry("600x500")
            self.root.configure(bg='lightgray')

            self.top_video_path = None
            self.bottom_video_path = None

            self.setup_ui()

        def setup_ui(self):
            """Create the GUI interface"""

            # Title
            title = tk.Label(
                self.root,
                text="🎬 Video Combiner",
                font=("Arial", 24, "bold"),
                bg="lightgray",
                fg="darkblue"
            )
            title.pack(pady=20)

            subtitle = tk.Label(
                self.root,
                text="Combine two videos: Top video above, Bottom video below",
                font=("Arial", 12),
                bg="lightgray",
                fg="gray"
            )
            subtitle.pack(pady=5)

            # Top video section
            tk.Label(
                self.root,
                text="📹 Step 1: Select TOP Video",
                font=("Arial", 14, "bold"),
                bg="lightgray"
            ).pack(pady=(30, 5))

            self.top_button = tk.Button(
                self.root,
                text="📁 Browse for Top Video",
                command=self.select_top_video,
                bg="lightblue",
                fg="black",
                font=("Arial", 12, "bold"),
                width=25,
                height=2,
                cursor="hand2"
            )
            self.top_button.pack(pady=5)

            self.top_status = tk.Label(
                self.root,
                text="❌ No top video selected",
                font=("Arial", 10),
                bg="lightgray",
                fg="red"
            )
            self.top_status.pack(pady=5)

            # Bottom video section
            tk.Label(
                self.root,
                text="📹 Step 2: Select BOTTOM Video",
                font=("Arial", 14, "bold"),
                bg="lightgray"
            ).pack(pady=(20, 5))

            self.bottom_button = tk.Button(
                self.root,
                text="📁 Browse for Bottom Video",
                command=self.select_bottom_video,
                bg="lightgreen",
                fg="black",
                font=("Arial", 12, "bold"),
                width=25,
                height=2,
                cursor="hand2"
            )
            self.bottom_button.pack(pady=5)

            self.bottom_status = tk.Label(
                self.root,
                text="❌ No bottom video selected",
                font=("Arial", 10),
                bg="lightgray",
                fg="red"
            )
            self.bottom_status.pack(pady=5)

            # Combine button
            self.combine_button = tk.Button(
                self.root,
                text="⚡ COMBINE VIDEOS",
                command=self.combine_videos,
                bg="orange",
                fg="white",
                font=("Arial", 16, "bold"),
                width=20,
                height=3,
                cursor="hand2",
                state="disabled"
            )
            self.combine_button.pack(pady=30)

            # Status label
            self.status_label = tk.Label(
                self.root,
                text="🔍 Please select both videos to continue",
                font=("Arial", 12),
                bg="lightgray",
                fg="blue"
            )
            self.status_label.pack(pady=10)

        def select_top_video(self):
            """Select top video file"""
            selected_file = filedialog.askopenfilename(
                title="Select TOP video file",
                filetypes=[
                    ("Video files", "*.mp4 *.avi *.mov *.mkv *.wmv *.flv"),
                    ("All files", "*.*")
                ]
            )

            if selected_file:
                self.top_video_path = selected_file
                filename = os.path.basename(selected_file)
                self.top_status.config(
                    text=f"✅ Top video: {filename}",
                    fg="green"
                )
                self.check_if_ready_to_combine()

        def select_bottom_video(self):
            """Select bottom video file"""
            selected_file = filedialog.askopenfilename(
                title="Select BOTTOM video file",
                filetypes=[
                    ("Video files", "*.mp4 *.avi *.mov *.mkv *.wmv *.flv"),
                    ("All files", "*.*")
                ]
            )

            if selected_file:
                self.bottom_video_path = selected_file
                filename = os.path.basename(selected_file)
                self.bottom_status.config(
                    text=f"✅ Bottom video: {filename}",
                    fg="green"
                )
                self.check_if_ready_to_combine()

        def check_if_ready_to_combine(self):
            """Enable combine button when both videos selected"""
            if self.top_video_path and self.bottom_video_path:
                self.combine_button.config(state="normal")
                self.status_label.config(
                    text="🚀 Ready to combine! Click the COMBINE button",
                    fg="green"
                )

        def combine_videos(self):
            """Combine the selected videos using GUI"""

            # Update status
            self.status_label.config(
                text="⏳ Processing videos... Please wait",
                fg="orange"
            )
            self.root.update()

            # Ask for output location
            output_path = filedialog.asksaveasfilename(
                title="Save combined video as...",
                defaultextension=".mp4",
                filetypes=[
                    ("MP4 files", "*.mp4"),
                    ("AVI files", "*.avi"),
                    ("All files", "*.*")
                ]
            )

            if output_path:
                # Use the command-line function to do the actual work
                success = combine_videos_cli(
                    self.top_video_path,
                    self.bottom_video_path,
                    output_path
                )

                if success:
                    self.status_label.config(
                        text="🎉 Video combined successfully!",
                        fg="green"
                    )
                    messagebox.showinfo(
                        "Success! 🎉",
                        f"Videos combined successfully!\n\nOutput saved to:\n{output_path}"
                    )
                else:
                    self.status_label.config(
                        text="❌ Error occurred during processing",
                        fg="red"
                    )
                    messagebox.showerror(
                        "Error! ❌",
                        "Failed to combine videos!\nCheck the console for details."
                    )
            else:
                self.status_label.config(
                    text="❌ Save cancelled by user",
                    fg="red"
                )

        def run(self):
            """Start the GUI application"""
            self.root.mainloop()

# ========================================
# MAIN PROGRAM ENTRY POINT
# ========================================

def main():
    """Main function that chooses between GUI and command-line mode"""

    print("=" * 60)
    print("🎬 SIMPLE VIDEO COMBINER")
    print("=" * 60)
    print("📦 Requirements: pip install moviepy")
    print("🎯 Purpose: Combine two videos in top-bottom layout")
    print("=" * 60)
    print("")

    # Check command line arguments
    if len(sys.argv) == 4:
        # Command line mode with arguments: script.py top_video bottom_video output_video
        top_video, bottom_video, output_video = sys.argv[1], sys.argv[2], sys.argv[3]
        print("🖥️ Running in command-line mode with provided arguments...")
        success = combine_videos_cli(top_video, bottom_video, output_video)
        if success:
            print(f"🎉 Success! Combined video saved as: {output_video}")
        else:
            print("❌ Failed to combine videos!")
        return

    # Try GUI mode first
    if GUI_AVAILABLE:
        try:
            print("🖥️ Attempting to start GUI mode...")
            app = SimpleCombiner()
            app.run()
            return
        except tk.TclError:
            print("❌ GUI failed to start (no display available)")
            print("🔄 Switching to command-line mode...")
        except Exception as e:
            print(f"❌ GUI error: {e}")
            print("🔄 Switching to command-line mode...")

    # Fall back to command-line mode
    print("💻 Running in command-line mode...")
    run_command_line_mode()

if __name__ == "__main__":
    main()

✅ MoviePy loaded successfully!
✅ GUI mode available
🎬 SIMPLE VIDEO COMBINER
📦 Requirements: pip install moviepy
🎯 Purpose: Combine two videos in top-bottom layout

🖥️ Attempting to start GUI mode...
❌ GUI Error: no display name and no $DISPLAY environment variable
🔄 Falling back to command-line mode...
❌ GUI failed to start (no display available)
🔄 Switching to command-line mode...
💻 Running in command-line mode...
🎬 VIDEO COMBINER - COMMAND LINE MODE
📹 Enter path to TOP video file: /content/YOLO11.mp4
📹 Enter path to BOTTOM video file: /content/YOLO12.mp4
💾 Enter output path (e.g., combined_video.mp4): 

📋 SUMMARY:
   Top video: /content/YOLO11.mp4
   Bottom video: /content/YOLO12.mp4
   Output: combined_video.mp4
Proceed? (y/n): y
🎬 Starting video combination process...
📂 Loading top video: YOLO11.mp4
📂 Loading bottom video: YOLO12.mp4
⏱️ Top video duration: 78.71 seconds
⏱️ Bottom video duration: 78.71 seconds
🎯 Using duration: 78.71 seconds
✂️ Trimming videos to same duration...
🔄 

t: 100%|██████████| 1890/1890 [07:53<00:00,  3.49it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/YOLO11.mp4, 3456000 bytes wanted but 0 bytes read,at frame 1889/1890, at time 78.71/78.71 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready combined_video.mp4
🧹 Cleaning up...
✅ Video combined successfully!

🎉 Success! Combined video saved as: combined_video.mp4
